In [1]:
import pandas as pd
from glob import glob


# Read single CSV file


In [2]:
single_file_acc=pd.read_csv("C:\data science\MetaMotion\A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")

In [3]:
single_file_gyr=pd.read_csv("C:\data science\MetaMotion\A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")

# List all data in data/raw/MetaMotion


In [4]:
files=glob("C:\data science\MetaMotion\*.csv")
len(files)

187

# Extract features from filename


In [5]:
data_path= "C:\data science\MetaMotion\\"
f=files[0]
participant =f.split("-")[0].replace(data_path," ")
print (participant)
label=f.split("-")[1]
print(label)
category=f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")
print(category)

df=pd.read_csv(f)
df["participant"]=participant
df["label"]=label
df["category"]=category

 A
bench
heavy


# Read all files

In [6]:
gyr_df=pd.DataFrame()
acc_df=pd.DataFrame()
acc_set=1
gyr_set=1
for f in files :
    participant =f.split("-")[0].replace(data_path," ")

    label=f.split("-")[1]

    category=f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")
    df=pd.read_csv(f)
    df["participant"]=participant
    df["label"]=label
    df["category"]=category
#split in diff for acc and gyr
    if "Accelerometer" in f:
        df["set"]=acc_set
        acc_set+=1
        acc_df=pd.concat([acc_df,df])
    if "Gyroscope"in f:
        df["set"]=gyr_set
        gyr_set+=1
        gyr_df=pd.concat([gyr_df,df])



# Working with datetimes

In [7]:
# Inspect the data structure
acc_df.info()

# Convert epoch milliseconds to datetime objects
pd.to_datetime(df["epoch (ms)"], unit="ms")

# Set the datetime as the index for both sensor DataFrames
acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit="ms")
gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit="ms")

# Delete redundant time columns to save memory
del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23578 entries, 0 to 241
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   epoch (ms)    23578 non-null  int64  
 1   time (01:00)  23578 non-null  object 
 2   elapsed (s)   23578 non-null  float64
 3   x-axis (g)    23578 non-null  float64
 4   y-axis (g)    23578 non-null  float64
 5   z-axis (g)    23578 non-null  float64
 6   participant   23578 non-null  object 
 7   label         23578 non-null  object 
 8   category      23578 non-null  object 
 9   set           23578 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 2.0+ MB


# Turn into function

In [8]:
files = glob("C:\data science\MetaMotion/*.csv")

def read_data_from_files(files):
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame()

    acc_set = 1
    gyr_set = 1

    for f in files:
        # Extract metadata from filename
        participant = f.split("-")[0].replace(data_path, "")
        label = f.split("-")[1]
        category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

        # Load the individual CSV
        df = pd.read_csv(f)

        # Add metadata columns to the dataframe
        df["participant"] = participant
        df["label"] = label
        df["category"] = category

        # Separate and label sets for Accelerometer data
        if "Accelerometer" in f:
            df["set"] = acc_set
            acc_set += 1
            acc_df = pd.concat([acc_df, df])

        # Separate and label sets for Gyroscope data
        if "Gyroscope" in f:
            df["set"] = gyr_set
            gyr_set += 1
            gyr_df = pd.concat([gyr_df, df])

    # Convert epoch to datetime index and clean up redundant columns
    acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit="ms")
    gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit="ms")

    del acc_df["epoch (ms)"]
    del acc_df["time (01:00)"]
    del acc_df["elapsed (s)"]

    del gyr_df["epoch (ms)"]
    del gyr_df["time (01:00)"]
    del gyr_df["elapsed (s)"]

    return acc_df, gyr_df

# Execute the function
acc_df, gyr_df = read_data_from_files(files)

# Merging datasets

In [9]:
data_merged = pd.concat([acc_df.iloc[:, :3], gyr_df], axis=1)
#axis 1 means vertical hai join


# Renaming the columns

In [10]:
data_merged.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
]

# Resample data (frequency conversion)
# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz


In [11]:
# 1. Define how to aggregate each column
# We want the average of sensor data, but the 'last' known label for categories
sampling = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last",
    "label": "last",
    
    "category": "last",
    "set": "last",
}

# 2. Split the master dataframe into a list of dataframes, one for each day
# This prevents resampling across the empty gaps at night
days = [g for n, g in data_merged.groupby(pd.Grouper(freq="D"))]

# 3. Use List Comprehension to resample each day individually
# .dropna() removes the empty rows created by pd.Grouper for days with no data
data_resampled = pd.concat([
    df.resample(rule="200ms").apply(sampling).dropna() 
    for df in days
])
data_resampled["set"]=data_resampled["set"].astype("int")
# 4. Final check of the new frequency
data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   int32  
dtypes: float64(6), int32(1), object(3)
memory usage: 739.0+ KB


# Export dataset

In [12]:
# Specify the filename at the end of the path
data_resampled.to_pickle(r"C:\data science\data-science-template-main\data\interim\01_data_processed.pkl")